# Question 2

The first task involves taking input from the user and ensuring that the correct movie is queried form the data base.  This method is the same for the script that queries the movies database in question 1.

I choose seven misspelled movie title to test my scripted with.

* "The poAtMan" - matches to "The Postman"

* "AVATor" - This one was used in class. I choose it because I wnated to ensure that could match it to both "Avatar" and "The Aviator".

* "Mysery mes" - matches to "Mystery Men"

* "liar laari" - matches to "Liar Liar"

* "raTatooy" - matches to "Ratatouille"

* "where harry met susan" - matches to "When Harry Met Sally..."

* "misses boubtfire" - matches to "Mrs. Doubtfire"

#### Import statements

In [1]:
import psycopg2 as psy

#### Database Connection

In [2]:
conn = psy.connect('dbname=imdb user=mtweed')

In [3]:
cur = conn.cursor()

#### Input
In order to standardize the input I usd string functions to ensure the capitalization was correct for the database.

In [4]:
query = input("Enter the name of a movie: ")

Enter the name of a movie: raTatooy


In [5]:
query = query.lower().title()

I found that I could be more selective of the results returned when matching a misspelled query if I omitted the word "the".

In [6]:
if "the" in query.lower().split():
    temp = query.lower().split()
    temp.remove('the')
    query = ' '.join(temp)
    query.title()

#### The SQL Matching Query

I used the similarity function for matching misspelled titles because it let me have much more control over how many results were returned as opposed to trigrams or ts_vectors. I found that a similarity value of 0.33 was optimal to return the desired values with as few other as possible. I also specified that only movies should be returned since the data is much more messy otherwise.

In [7]:
cur.execute("""
            select distinct primarytitle
            from title_basics
            where similarity(primarytitle, %s) > 0.33 and titletype = 'movie';
            """, (query,))
results = cur.fetchall()
results = [results[x][0] for x in range(len(results))]

To further limit the number of title that are proposed for the user's selection, I used the levenshtein function to calculate the Levenshein numnber.  Again, I had to adjest the values to enure the the desired results were returned without too many undesired ones.  I set the score so that there was no penalty for replacements, a penalty on one for deletion, and a penalty of two for insertions.  These limits were found through trial and error. I set the cutoff threshold at 5 because, this values allowed for the addition of 'the' if necessary.

In [8]:
dist = []
for q in results:
    cur.execute("select levenshtein(%s, %s,2,1,0);", (q, query,))
    temp = cur.fetchall()
    dist.append(temp)

In [9]:
dist = [x[0][0] for x in dist]

In [10]:
pos = [results[x] for x in range(len(dist)) if dist[x] <= 5]

#### User Varification of Input

This script asks the user to select their intended query from a list. It shows ten possibilities at a time until the correct title is shown.

In [11]:
while True:
    opt = []
    if len(pos) == 0:
        print("Nothing found with that name.")
        break
        
    print("\nDid you mean:")
    if len(pos) > 10:
        for i in range(10):
            opt.append(pos.pop())
            print(str(i) + ") "+opt[i])
    else:
        for i in range(len(pos)):
            opt.append(pos.pop())
            print(str(i) + ") "+opt[i])
            
    sel = input("Make selection or press ENTER for more")
    
    if sel == '':
        continue
    else:
        sel = int(sel)
        
    if sel in range(10):
        query = opt[sel]
        break
    elif len(opt)==0:
        break
    else:
        continue


Did you mean:
0) Ratboy
1) RatAttack
2) Ratatouille
3) Ratataplan
Make selection or press ENTER for more2


## Matching The Query's Genre
Since there are many more entries in the imdb database, queries bases on primarytitle often results in several titles being returned.  I first had to write this scrip that further clarified what movie the user intended.  This script gathers the original title and year so that only a single entry's genre is returned.

In [12]:
cur.execute("""
            select genres, originaltitle, startyear 
            from title_basics 
            where primarytitle = %s and titletype = 'movie'
            """
            , [query])

In [13]:
q_genres = cur.fetchall()

In [14]:
if len(q_genres) > 1:
    orig_title = [x[1] for x in q_genres]
    year = [x[2] for x in q_genres]
    q_genres = [x[0] for x in q_genres]
    choice = {}
    for i in range(len(q_genres)):
        choice[i] = q_genres[i]
    while True:
        print('Did you mean:\n')
        for i in range(len(q_genres)):
            print(str(list(choice.keys())[i]+1) + ') ' + orig_title[i]+' from '+str(year[i])+'?')
        ans = int(input("Selection: "))

        if ans-1 in list(choice.keys()):
            q_genres = [x for x in choice[ans-1]]
            break
        else:
            print("Invalid response.")
else:
    q_genres = [x[0][y] for x in q_genres for y in range(len(x[0]))]

I used this query to select only the titles that had genres that were a superset (using the @> operator) of the genres of the query. This way, the titles returned would at least be representative of every genre that the query represented.  Due to the size of the database, this query often resulted in many titles.  To help organize this information, I though it would be best to orger the data by the average rating (highest rated first) followed by the number of votes (most votes first).  This way the most well known and highest rated movies are returned first.

In [15]:
cur.execute("""
            select tb.primarytitle, tr.averagerating rat, tr.numvotes vot
            from title_basics tb, title_ratings tr 
            where genres @> %s and tr.titleid = tb.titleid and titletype = 'movie' and primarytitle != %s
            order by vot desc, rat desc;
            """, [q_genres, query])

In [16]:
results = cur.fetchall()

In [17]:
results = [x[0] for x in results]

In [18]:
print("The movies closes in genre to {} are listed below in order of average rating\n".format(query))
while True:
    if len(results) > 10:
        for i in range(10):
            temp = results.pop(0)
            print(temp)
    else:
        for j in range(len(results)):
            print(temp)
            break
    quit = input("Press ENTER for more\nOr q to quit\n")
    if quit.lower() == 'q':
        break

The movies closes in genre to Ratatouille are listed below in order of average rating

Finding Nemo
Up
Toy Story
Monsters, Inc.
Toy Story 3
Shrek
Inside Out
Frozen
Toy Story 2
Ice Age
Press ENTER for more
Or q to quit

Zootopia
Shrek 2
Tangled
Wreck-It Ralph
Despicable Me 2
Brave
Madagascar
Aladdin
Monsters University
The Simpsons Movie
Press ENTER for more
Or q to quit

Shrek the Third
A Bug's Life
Ice Age: The Meltdown
Moana
Rango
Finding Dory
Ice Age: Dawn of the Dinosaurs
Minions
Rio
Cloudy with a Chance of Meatballs
Press ENTER for more
Or q to quit
q


In [19]:
conn.close()

## Example Console Output

NOTE: This was run on my local testing server because the nessesary extensions were missing on cs1.

```
Enter the name of a movie: The poAtMan

Did you mean:
0) The Postman
1) The Boatman
2) Postman
3) Portman
4) Mr. Patman
5) Fatman
6) Boatman
7) Batman
8) Atman
Make selection or press ENTER for more0
Did you mean:

1) Postchi from 1972?
2) The Postman from 1997?
3) Al-boustaguy from 1968?
4) El bostagi from 1948?
5) The Postman from 0?
Selection: 2
The movies closes in genre to The Postman are listed below in order of average rating

Gladiator
Avengers: Endgame
Dawn of the Planet of the Apes
King Kong
Seven Samurai
The Book of Eli
Apocalypto
Snow White and the Huntsman
Kingdom of Heaven
Noah
Press ENTER for more
Or q to quit

The Grey
The Twilight Saga: Breaking Dawn - Part 1
War for the Planet of the Apes
San Andreas
Master and Commander: The Far Side of the World
After Earth
King Arthur: Legend of the Sword
Robin Hood: Prince of Thieves
The Promise
The Legend of Tarzan
Press ENTER for more
Or q to quit
q
(base) [mtweed@miles-alienwarem17r3 Database_py_scripts]$ ./Q2_imdb.py 
Enter the name of a movie: AVATor

Did you mean:
0) Vator
1) The Aviator
2) Salvator
3) Her Aviator
4) Hellavator
5) Elevator
6) Ekvator
7) Aviator
8) Avataran
9) Avatara
Make selection or press ENTER for more

Did you mean:
0) Avatar 5
1) Avatar 4
2) Avatar 3
3) Avatar 2
4) Avatar
5) Avas
6) Avan
7) Avam
8) Aval
9) Animator
Make selection or press ENTER for more4
Did you mean:

1) Avatar from 1941?
2) Avatar from 1916?
3) Avatar from 2009?
4) Abatâ from 2011?
Selection: 3
The movies closes in genre to Avatar are listed below in order of average rating

Star Wars: Episode IV - A New Hope
Star Wars: Episode V - The Empire Strikes Back
Pirates of the Caribbean: The Curse of the Black Pearl
Star Wars: Episode VI - Return of the Jedi
Thor
Star Wars: Episode I - The Phantom Menace
Star Wars: Episode III - Revenge of the Sith
Pirates of the Caribbean: Dead Man's Chest
Star Wars: Episode II - Attack of the Clones
Pirates of the Caribbean: At World's End
Press ENTER for more
Or q to quit

Thor: The Dark World
Suicide Squad
Doctor Strange
Wonder Woman
Star Wars: Episode VIII - The Last Jedi
Pirates of the Caribbean: On Stranger Tides
The Mummy
Justice League
Aquaman
Fantastic Four
Press ENTER for more
Or q to quit
q
(base) [mtweed@miles-alienwarem17r3 Database_py_scripts]$ ./Q2_imdb.py 
Enter the name of a movie: Mysery mes

Did you mean:
0) Mystery Men
1) Mystery Files
2) Mes Mer...
3) Me... Myself
Make selection or press ENTER for more0
The movies closes in genre to Mystery Men are listed below in order of average rating

Scott Pilgrim vs. the World
Ghostbusters
Ghostbusters
Ghostbusters II
Kung Fu Hustle
Shaolin Soccer
Pride and Prejudice and Zombies
Bulletproof Monk
Buffy the Vampire Slayer
The Medallion
Press ENTER for more
Or q to quit

The Monster Squad
Eega
Vampire Killers
Witching and Bitching
Undead
Karan Arjun
Yoga Hosers
A Chinese Ghost Story
The God of Cookery
Golmaal Again
Press ENTER for more
Or q to quit
q
(base) [mtweed@miles-alienwarem17r3 Database_py_scripts]$ ./Q2_imdb.py 
Enter the name of a movie: liar laari

Did you mean:
0) The Liar
1) Some Liar
2) Liar Liar Wolf
3) Liar Liar
4) Liar.Com
5) King Liar
Make selection or press ENTER for more3
The movies closes in genre to Liar Liar are listed below in order of average rating

Groundhog Day
Monty Python and the Holy Grail
Midnight in Paris
Bruce Almighty
Scott Pilgrim vs. the World
Ghostbusters
The Mask
Being John Malkovich
Click
The Secret Life of Walter Mitty
Press ENTER for more
Or q to quit
q
(base) [mtweed@miles-alienwarem17r3 Database_py_scripts]$ ./Q2_imdb.py 
Enter the name of a movie: raTatooy

Did you mean:
0) Ratboy
1) RatAttack
2) Ratatouille
3) Ratataplan
Make selection or press ENTER for more2
The movies closes in genre to Ratatouille are listed below in order of average rating

Finding Nemo
Up
Toy Story
Monsters, Inc.
Toy Story 3
Shrek
Inside Out
Frozen
Toy Story 2
Ice Age
Press ENTER for more
Or q to quit

Zootopia
Shrek 2
Tangled
Wreck-It Ralph
Despicable Me 2
Brave
Madagascar
Aladdin
Monsters University
The Simpsons Movie
Press ENTER for more
Or q to quit
q
(base) [mtweed@miles-alienwarem17r3 Database_py_scripts]$ ./Q2_imdb.py 
Enter the name of a movie: where harry met susan

Did you mean:
0) When Harry Met Sally...
1) Jab Harry Met Sejal
Make selection or press ENTER for more0
The movies closes in genre to When Harry Met Sally... are listed below in order of average rating

Silver Linings Playbook
Life Is Beautiful
500 Days of Summer
Crazy, Stupid, Love.
Love Actually
The Terminal
Easy A
50 First Dates
Up in the Air
50/50
Press ENTER for more
Or q to quit

Moonrise Kingdom
The Proposal
10 Things I Hate About You
As Good as It Gets
Notting Hill
Forgetting Sarah Marshall
The Graduate
Vicky Cristina Barcelona
Jerry Maguire
The Artist
Press ENTER for more
Or q to quit
q
(base) [mtweed@miles-alienwarem17r3 Database_py_scripts]$ ./Q2_imdb.py 
Enter the name of a movie: misses boubtfire

Did you mean:
0) Mrs. Doubtfire
Make selection or press ENTER for more0
The movies closes in genre to Mrs. Doubtfire are listed below in order of average rating

Modern Times
Marley & Me
We Bought a Zoo
Babe
The Kid
The Santa Clause
A League of Their Own
The Sandlot
My Girl
Beethoven
Press ENTER for more
Or q to quit

What a Girl Wants
The Mighty Ducks
High School Musical 3
Secondhand Lions
3 Men and a Baby
The Muppet Christmas Carol
The Odd Life of Timothy Green
Diary of a Wimpy Kid
Hannah Montana: The Movie
Miracle on 34th Street
Press ENTER for more
Or q to quit
q

```